In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import preprocessing

# Corpos de Prova

In [2]:
corpos_prova = pd.read_csv("ResultadosPorNotaFiscal.csv", index_col='CodigoNotaFiscal')
corpos_prova.head()

,Idade,DataRompimento,TensaoRompimento,Status
CodigoNotaFiscal,,,,
2529,3,2010-07-19,NaN,A
2529,3,2010-07-19,NaN,A
3804,7,2010-07-30,NaN,A
8343,7,2010-08-23,NaN,A
8347,7,2010-08-23,NaN,A


In [3]:
corpos_prova.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804166 entries, 2529 to 100644
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Idade             804166 non-null  int64  
 1   DataRompimento    804166 non-null  object 
 2   TensaoRompimento  666030 non-null  float64
 3   Status            804166 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 30.7+ MB


In [4]:
corpos_prova.describe()

,Idade,TensaoRompimento
count,804166.000000,666030.000000
mean,20.348777,31.341119
std,10.273401,9.366419
min,1.000000,1.430000
25%,7.000000,24.840000
50%,28.000000,30.860000
75%,28.000000,37.310000
max,60.000000,406.740000


# Nota Fiscal

In [5]:
notas_fiscais = pd.read_csv("NotaFiscal.csv")
notas_fiscais.head()

,CodigoNotaFiscal,CodigoCentroSaida,CodigoReceita,Volume,RelacaoAguaCimento,TipoLancamento,DataEmissaoNotaFiscal,CodigoCliente,CodigoMotorista,Status,QuantidadeAgua,QuantidadeAdicaoAgua,TempoPermanenciaObra,ConsumoTotal
0,19,79020,11051,7.0,4.065,C,2010-07-02 06:13:47,11671051,56079,C,931,NaN,NaN,2478.0
1,22,51020,11051,7.0,4.065,C,2010-07-02 06:29:25.063,11671051,173051,N,1085,NaN,2010-07-02 01:43:00,2478.0
2,23,51020,15051,7.0,3.596,C,2010-07-02 06:37:31.09,12376051,66079,N,1092,NaN,2010-07-02 00:50:00,2800.0
3,25,51020,80051,5.0,3.136,C,2010-07-02 06:51:52.637,11953051,193051,N,885,NaN,2010-07-02 01:40:00,1315.0
4,32,79020,574051,8.0,2.795,B,2010-07-02 07:06:20.307,11579051,23079,N,1328,NaN,2010-07-02 02:35:00,4480.0


In [6]:
notas_fiscais.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554726 entries, 0 to 554725
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CodigoNotaFiscal       554726 non-null  int64  
 1   CodigoCentroSaida      554726 non-null  int64  
 2   CodigoReceita          554726 non-null  int64  
 3   Volume                 554726 non-null  float64
 4   RelacaoAguaCimento     554726 non-null  float64
 5   TipoLancamento         554726 non-null  object 
 6   DataEmissaoNotaFiscal  554726 non-null  object 
 7   CodigoCliente          554726 non-null  int64  
 8   CodigoMotorista        554726 non-null  int64  
 9   Status                 554726 non-null  object 
 10  QuantidadeAgua         554726 non-null  int64  
 11  QuantidadeAdicaoAgua   6033 non-null    float64
 12  TempoPermanenciaObra   538829 non-null  object 
 13  ConsumoTotal           554725 non-null  float64
dtypes: float64(4), int64(6), object(4)
m

*Removendo notas fiscais que tiveram adição de água ou que não tem consumo*

In [7]:
mask_not_null = notas_fiscais.notna()

mask_quantidade_agua_null = mask_not_null["QuantidadeAdicaoAgua"] == False
mask_consumo_total_not_null = mask_not_null["ConsumoTotal"]

notas_fiscais = notas_fiscais[mask_quantidade_agua_null & mask_consumo_total_not_null]

In [8]:
mask_notas_nao_canceladas = notas_fiscais["Status"] == 'N'

notas_fiscais = notas_fiscais[mask_notas_nao_canceladas]

# Traço

In [9]:
tracos = pd.read_csv("InformacoesReceita.csv", index_col = 'CodigoReceita')
tracos.head()

,Idade,TipoTraco,CodigoAglomerante,FCK,RelacaoAguaCimento
CodigoReceita,,,,,
64051,28,N,3051.0,40.0,0.472
65051,28,N,8051.0,40.0,0.534
66051,28,N,41051.0,40.0,0.511
67051,28,N,12051.0,40.0,0.529
2750051,28,N,18051.0,40.0,0.504


In [10]:
mask_idade = tracos["Idade"] == 28

tracos = tracos[mask_idade]

# Traco Insumo

In [11]:
insumos_traco = pd.read_csv("ItemNotaFiscal.csv")
insumos_traco.head()

,CodigoNotaFiscal,TipoInsumo,CodigoInsumo,CodigoFornecedor
0,19,C,13,205841
1,19,E,9,205841
2,22,C,13,205841
3,22,E,9,205841
4,23,C,13,205841


## Inserindo informações na nota fiscal

In [12]:
notas_fiscais.head()

,CodigoNotaFiscal,CodigoCentroSaida,CodigoReceita,Volume,RelacaoAguaCimento,TipoLancamento,DataEmissaoNotaFiscal,CodigoCliente,CodigoMotorista,Status,QuantidadeAgua,QuantidadeAdicaoAgua,TempoPermanenciaObra,ConsumoTotal
1,22,51020,11051,7.0,4.065,C,2010-07-02 06:29:25.063,11671051,173051,N,1085,NaN,2010-07-02 01:43:00,2478.0
2,23,51020,15051,7.0,3.596,C,2010-07-02 06:37:31.09,12376051,66079,N,1092,NaN,2010-07-02 00:50:00,2800.0
3,25,51020,80051,5.0,3.136,C,2010-07-02 06:51:52.637,11953051,193051,N,885,NaN,2010-07-02 01:40:00,1315.0
4,32,79020,574051,8.0,2.795,B,2010-07-02 07:06:20.307,11579051,23079,N,1328,NaN,2010-07-02 02:35:00,4480.0
6,34,51020,104051,7.0,3.593,B,2010-07-02 07:17:47.17,11190051,377051,N,1246,NaN,2010-07-02 00:30:00,3178.0


In [13]:
dados = notas_fiscais.merge(tracos, on="CodigoReceita", suffixes=(" Emitida", " Receita"))

mask_ac_minimo = dados["RelacaoAguaCimento Emitida"] >= 0.4
mask_ac_maximo = dados["RelacaoAguaCimento Emitida"] <= 1.3

dados = dados[mask_ac_minimo & mask_ac_maximo]

dados.head()

,CodigoNotaFiscal,CodigoCentroSaida,CodigoReceita,Volume,RelacaoAguaCimento Emitida,TipoLancamento,DataEmissaoNotaFiscal,CodigoCliente,CodigoMotorista,Status,QuantidadeAgua,QuantidadeAdicaoAgua,TempoPermanenciaObra,ConsumoTotal,Idade,TipoTraco,CodigoAglomerante,FCK,RelacaoAguaCimento Receita
1,127,51120,11051,8.0,1.011,C,2010-07-03 09:19:34.263,12403051,465051,N,1240,NaN,2010-07-03 00:20:00,2832.0,28,N,3051.0,15.0,0.9
2,149,51120,11051,8.0,1.011,C,2010-07-03 10:54:43.287,12403051,193051,N,1240,NaN,2010-07-03 00:26:00,2832.0,28,N,3051.0,15.0,0.9
3,165,51120,11051,8.0,1.011,C,2010-07-03 12:18:12.393,12403051,3060,N,1240,NaN,2010-07-03 00:28:00,2832.0,28,N,3051.0,15.0,0.9
4,208,51020,11051,8.0,1.011,C,2010-07-05 06:53:43.77,11575051,518051,N,1240,NaN,2010-07-05 00:30:00,2832.0,28,N,3051.0,15.0,0.9
5,214,51020,11051,5.0,1.011,C,2010-07-05 07:18:10.7,12418051,443051,N,775,NaN,2010-07-05 00:20:00,1770.0,28,N,3051.0,15.0,0.9


In [14]:
corpos_prova_notas_selecionadas = pd.DataFrame(data = dados["CodigoNotaFiscal"]).merge(corpos_prova, on = "CodigoNotaFiscal")
corpos_prova_notas_selecionadas.set_index("CodigoNotaFiscal", inplace=True)
corpos_prova_notas_selecionadas.head()

,Idade,DataRompimento,TensaoRompimento,Status
CodigoNotaFiscal,,,,
208,7,2010-07-12,8.78,C
208,28,2010-08-02,17.68,C
208,28,2010-08-02,18.33,C
234,7,2010-07-12,8.13,C
234,28,2010-08-02,19.67,C


In [15]:
mask_notas_com_rompimento = dados["CodigoNotaFiscal"].isin(corpos_prova_notas_selecionadas.index)

dados_notas_com_rompimento = dados[mask_notas_com_rompimento]
dados_notas_com_rompimento.head()

,CodigoNotaFiscal,CodigoCentroSaida,CodigoReceita,Volume,RelacaoAguaCimento Emitida,TipoLancamento,DataEmissaoNotaFiscal,CodigoCliente,CodigoMotorista,Status,QuantidadeAgua,QuantidadeAdicaoAgua,TempoPermanenciaObra,ConsumoTotal,Idade,TipoTraco,CodigoAglomerante,FCK,RelacaoAguaCimento Receita
4,208,51020,11051,8.0,1.011,C,2010-07-05 06:53:43.77,11575051,518051,N,1240,NaN,2010-07-05 00:30:00,2832.0,28,N,3051.0,15.0,0.9
6,234,51020,11051,8.0,1.011,C,2010-07-05 08:09:08.35,11575051,409051,N,1240,NaN,2010-07-05 00:45:00,2832.0,28,N,3051.0,15.0,0.9
11,365,51120,11051,8.0,1.011,C,2010-07-05 14:58:29.897,8547051,465051,N,1184,NaN,2010-07-05 00:45:00,2832.0,28,N,3051.0,15.0,0.9
12,416,51020,11051,5.0,1.011,C,2010-07-06 06:10:47.513,11344051,193051,N,740,NaN,2010-07-06,1770.0,28,N,3051.0,15.0,0.9
13,436,51120,11051,8.0,1.011,C,2010-07-06 07:19:22.543,11575051,49079,N,1184,NaN,2010-07-06,2832.0,28,N,3051.0,15.0,0.9


In [16]:
def EncontraRompimento7Dias(nota):
    rompimentos_nota_fiscal = corpos_prova_notas_selecionadas.loc[nota["CodigoNotaFiscal"]]
    
    mask_minimo_7_dias = rompimentos_nota_fiscal["Idade"] >= 7
    mask_maximo_7_dias = rompimentos_nota_fiscal["Idade"] <= 8
    
    rompimentos_7_dias = rompimentos_nota_fiscal[mask_minimo_7_dias & mask_maximo_7_dias]
    
    if (len(rompimentos_7_dias) == 0 or type(rompimentos_7_dias) == str):
        return np.NaN;
    
    rompimentos_7_dias = rompimentos_7_dias.sort_values("TensaoRompimento", )
    
    if (rompimentos_7_dias.iloc[len(rompimentos_7_dias) - 1]["Status"] != 'R'):
        return np.NaN;
    
    return rompimentos_7_dias.iloc[len(rompimentos_7_dias) - 1]["TensaoRompimento"]

def EncontraRompimento28Dias(nota):
    rompimentos_nota_fiscal = corpos_prova_notas_selecionadas.loc[nota["CodigoNotaFiscal"]]
    
    mask_minimo_28_dias = rompimentos_nota_fiscal["Idade"] >= 28
    mask_maximo_28_dias = rompimentos_nota_fiscal["Idade"] <= 31
    
    rompimentos_28_dias = rompimentos_nota_fiscal[mask_minimo_28_dias & mask_maximo_28_dias]
    
    if (type(rompimentos_28_dias) == np.int64 or len(rompimentos_28_dias) == 0 or type(rompimentos_28_dias) == str):
        return np.NaN;
    
    rompimentos_28_dias = rompimentos_28_dias.sort_values("TensaoRompimento")
    
    if (rompimentos_28_dias.iloc[len(rompimentos_28_dias) - 1]["Status"] != 'R'):
        return np.NaN;
    
    return rompimentos_28_dias.iloc[len(rompimentos_28_dias) - 1]["TensaoRompimento"]

In [17]:
dados_notas_com_rompimento["Rompimento7Dias"] = dados_notas_com_rompimento.apply(EncontraRompimento7Dias, axis = "columns")

<ipython-input-17-930e365ea765>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_notas_com_rompimento["Rompimento7Dias"] = dados_notas_com_rompimento.apply(EncontraRompimento7Dias, axis = "columns")


In [18]:
dados_notas_com_rompimento["Rompimento28Dias"] = dados_notas_com_rompimento.apply(EncontraRompimento28Dias, axis = "columns")

<ipython-input-18-2db96d85f9f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_notas_com_rompimento["Rompimento28Dias"] = dados_notas_com_rompimento.apply(EncontraRompimento28Dias, axis = "columns")


In [19]:
dados_notas_com_rompimento["Rompimento7Dias"].notna()

4         False
6         False
11         True
12        False
13         True
          ...  
410754    False
410758    False
410759    False
410761    False
410763    False
Name: Rompimento7Dias, Length: 199703, dtype: bool

In [20]:
mask_7_dias = dados_notas_com_rompimento["Rompimento7Dias"].notna()
mask_28_dias = dados_notas_com_rompimento["Rompimento28Dias"].notna()

dados_com_rompimento = dados_notas_com_rompimento[mask_7_dias & mask_28_dias]

In [22]:
item_nota_fiscal = pd.read_csv("ItemNotaFiscal.csv")

mask_cimento = item_nota_fiscal["TipoInsumo"] == 'C'
item_nota_fiscal = item_nota_fiscal[mask_cimento]
item_nota_fiscal.drop("TipoInsumo", axis = 1, inplace = True)

item_nota_fiscal.head()

,CodigoNotaFiscal,CodigoInsumo,CodigoFornecedor
0,19,13,205841
2,22,13,205841
4,23,13,205841
6,25,13,205841
8,32,13,205841


In [24]:
dados_com_rompimento = dados_com_rompimento.merge(item_nota_fiscal, on="CodigoNotaFiscal")

In [27]:
dados_com_rompimento.columns = ['CodigoNotaFiscal', 'CodigoCentroSaida', 'CodigoReceita', 'Volume',
       'RelacaoAguaCimento Emitida', 'TipoLancamento', 'DataEmissaoNotaFiscal',
       'CodigoCliente', 'CodigoMotorista', 'Status', 'QuantidadeAgua',
       'QuantidadeAdicaoAgua', 'TempoPermanenciaObra', 'ConsumoTotal', 'Idade',
       'TipoTraco', 'CodigoAglomerante', 'FCK', 'RelacaoAguaCimento Receita',
       'Rompimento7Dias', 'Rompimento28Dias', 'CodigoCimento',
       'CodigoFornecedorCimento']

In [28]:
mask_codigo_cimento_existente = dados_com_rompimento["CodigoCimento"].notna()

dados_com_rompimento = dados_com_rompimento[mask_codigo_cimento_existente]

In [29]:
mask_codigo_aglomerante_existente = dados_com_rompimento["CodigoAglomerante"].notna()

dados_com_rompimento = dados_com_rompimento[mask_codigo_aglomerante_existente]

In [30]:
curvas_laboratorio = pd.read_csv("HistoricoCurvaAbramsEDesvios.csv",  index_col = "CodigoAglomerante")
curvas_laboratorio.head()

,Idade,AbramsX,AbramsY,FCKFaixa1,DesvioFaixa1,DesvioTEFaixa1,FCKFaixa2,DesvioFaixa2,DesvioTEFaixa2,FCKFaixa3,DesvioFaixa3,DesvioTEFaixa3,FCKFaixa4,DesvioFaixa4,DesvioTEFaixa4,DesvioMaximo,DesvioTEMaximo,DataAlteracao
CodigoAglomerante,,,,,,,,,,,,,,,,,,
13051,0,1.4763,103.500,18,5.0,NaN,25.0,4.5,NaN,34.0,4.2,NaN,NaN,NaN,NaN,5.0,NaN,2010-06-30 16:55:51.217
13051,28,-1.4763,103.500,18,5.0,NaN,25.0,4.5,NaN,34.0,4.2,NaN,NaN,NaN,NaN,5.0,NaN,2010-06-30 16:55:51.233
14051,0,-1.3574,81.545,20,3.0,NaN,25.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,2010-06-30 17:05:06.983
14051,28,-1.3574,81.545,20,3.0,NaN,25.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,2010-06-30 17:05:07
3051,0,-1.4763,103.500,18,5.0,NaN,25.0,4.5,NaN,34.0,4.2,NaN,NaN,NaN,NaN,5.0,NaN,2010-06-30 18:47:15.397


In [31]:
def EncontrarResultadoEsperadoNotaFiscal(nota):
    curvas_do_aglomerante = curvas_laboratorio.loc[nota["CodigoAglomerante"]]
    
    mask_idade_28 = curvas_do_aglomerante["Idade"] == 28
    curvas_28 = curvas_do_aglomerante[mask_idade_28]
    
    mask_data_emissao_nota = curvas_28["DataAlteracao"] <= nota["DataEmissaoNotaFiscal"]
    curvas_do_aglomerante_antes_emissao_nota = curvas_28[mask_data_emissao_nota]
    curvas_do_aglomerante_antes_emissao_nota = curvas_do_aglomerante_antes_emissao_nota.sort_values("DataAlteracao")
    
    if (curvas_do_aglomerante_antes_emissao_nota.empty):
        return np.NaN;
    
    curva_laboratorio_momento = curvas_do_aglomerante_antes_emissao_nota.iloc[len(curvas_do_aglomerante_antes_emissao_nota) - 1]    
    resultado_esperado = curva_laboratorio_momento["AbramsY"] * np.exp(curva_laboratorio_momento["AbramsX"] * nota["RelacaoAguaCimento Emitida"])
    
    return resultado_esperado;

In [32]:
dados_com_rompimento["ResultadoEsperado"] = dados_com_rompimento.apply(EncontrarResultadoEsperadoNotaFiscal, axis = "columns")

In [33]:
dados_com_rompimento["DataEmissaoNotaFiscal"] = pd.to_datetime(dados_com_rompimento["DataEmissaoNotaFiscal"])

In [34]:
curvas_crescimento = pd.read_csv("HistoricoCurvasCrescimento.csv", index_col="CodigoAglomerante")
curvas_crescimento["DataAlteracao"] = pd.to_datetime(curvas_crescimento["DataAlteracao"])
curvas_crescimento.head()

,Ano,Mes,ParametroX,ParametroY,DataAlteracao
CodigoAglomerante,,,,,
3051,2010,6,2.4050,-0.282,2010-07-09 08:47:36.923
3051,2010,6,2.4050,-0.282,2010-07-09 08:49:22.373
3051,2010,7,2.4050,-0.282,2010-07-09 08:53:40.973
4051,2010,7,1.9276,-0.183,2010-07-09 08:56:44.683
5051,2010,7,2.5370,-0.322,2010-07-09 08:57:42.057


In [35]:
def EncontrarProjecaoNotaFiscal(nota):
    curvas_do_aglomerante = curvas_crescimento.loc[nota["CodigoAglomerante"]]
    
    mask_mes = curvas_do_aglomerante["Mes"] == nota["DataEmissaoNotaFiscal"].month
    mask_ano = curvas_do_aglomerante["Ano"] == nota["DataEmissaoNotaFiscal"].year
    
    curvas_mes_ano_corretos = curvas_do_aglomerante[mask_mes & mask_ano]
    
    mask_data_emissao_nota = curvas_mes_ano_corretos["DataAlteracao"] <= nota["DataEmissaoNotaFiscal"]
    curvas_antes_emissao_nota = curvas_mes_ano_corretos[mask_data_emissao_nota]
    curvas_antes_emissao_nota = curvas_antes_emissao_nota.sort_values("DataAlteracao")
    
    if (curvas_antes_emissao_nota.empty):
        return np.NaN;
    
    curva_crescimento_momento = curvas_antes_emissao_nota.iloc[len(curvas_antes_emissao_nota) - 1]
    crescimento = ((curva_crescimento_momento["ParametroY"] * np.log(nota["Rompimento7Dias"])) + curva_crescimento_momento["ParametroX"] - 1)
    resultado_projetado = (crescimento * nota["Rompimento7Dias"]) + nota["Rompimento7Dias"]
    
    return resultado_projetado;

In [36]:
dados_com_rompimento["ResultadoProjetado"] = dados_com_rompimento.apply(EncontrarProjecaoNotaFiscal, axis = "columns")

In [37]:
mask_tem_projecao = dados_com_rompimento["ResultadoProjetado"].notna()

dados_com_rompimento = dados_com_rompimento[mask_tem_projecao]

In [38]:
def EncontraModuloDiferencaEntreRealProjetado(nota):
    diferenca = (nota["Rompimento28Dias"] - nota["ResultadoProjetado"])/nota["Rompimento28Dias"]
    return np.abs(diferenca);

In [39]:
dados_com_rompimento["DiferencaRealProjetado"] = dados_com_rompimento.apply(EncontraModuloDiferencaEntreRealProjetado, axis = "columns")

In [40]:
dados_com_rompimento.drop(["CodigoNotaFiscal", "CodigoCentroSaida", "Volume", "DataEmissaoNotaFiscal", "CodigoCliente", "CodigoMotorista", "Status", "QuantidadeAdicaoAgua", "TempoPermanenciaObra"], axis=1, inplace=True)
dados_com_rompimento.head()

,CodigoReceita,RelacaoAguaCimento Emitida,TipoLancamento,QuantidadeAgua,ConsumoTotal,Idade,TipoTraco,CodigoAglomerante,FCK,RelacaoAguaCimento Receita,Rompimento7Dias,Rompimento28Dias,CodigoCimento,CodigoFornecedorCimento,ResultadoEsperado,ResultadoProjetado,DiferencaRealProjetado
2,11051,1.011,C,930,2124.0,28,N,3051.0,15.0,0.9,10.50,18.37,13,205841,23.266913,18.868387,0.027130
3,11051,1.011,C,1240,2832.0,28,N,3051.0,15.0,0.9,11.69,19.55,13,205841,23.266913,20.723169,0.060009
4,11051,1.011,C,1240,2832.0,28,N,3051.0,15.0,0.9,10.16,16.73,13,205841,23.266913,18.332992,0.095815
5,11051,1.011,C,1240,2832.0,28,N,3051.0,15.0,0.9,13.52,17.19,13,205841,23.266913,23.522874,0.368405
6,11051,1.011,C,1240,2832.0,28,N,3051.0,15.0,0.9,7.71,13.52,13,205841,23.266913,14.392958,0.064568


In [41]:
dados_com_rompimento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125292 entries, 2 to 142468
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   CodigoReceita               125292 non-null  int64  
 1   RelacaoAguaCimento Emitida  125292 non-null  float64
 2   TipoLancamento              125292 non-null  object 
 3   QuantidadeAgua              125292 non-null  int64  
 4   ConsumoTotal                125292 non-null  float64
 5   Idade                       125292 non-null  int64  
 6   TipoTraco                   125292 non-null  object 
 7   CodigoAglomerante           125292 non-null  float64
 8   FCK                         125292 non-null  float64
 9   RelacaoAguaCimento Receita  125292 non-null  float64
 10  Rompimento7Dias             125292 non-null  float64
 11  Rompimento28Dias            125292 non-null  float64
 12  CodigoCimento               125292 non-null  int64  
 13  CodigoForneced

In [42]:
dados_com_rompimento.to_csv("dados_pre_processados.csv")